Target Specification:
- OS: Ubuntu 22.04.4 LTS
- Python: 3.11
- Platform: Kaggle
- CPU: Intel Xeon 8173M 2 Core / 4 Thread
- RAM: 30GB
- GPU: 2* Nvidia T4 15GB

Pre Training Initialization

In [ ]:
from pathlib import Path
%pip install onnxruntime-gpu pytorch-msssim
""" if not Path("/kaggle/working/nv-tensorrt-local-repo-ubuntu2204-10.9.0-cuda-12.8_1.0-1_amd64.deb").exists():
    !sudo apt remove --purge -y 'libnvinfer*' 'tensorrt*' 'python3-libnvinfer*'
    !sudo apt autoremove -y
    !sudo apt clean
    !sudo rm -rf /var/lib/apt/lists/*
    %pip install onnxruntime-gpu pytorch-msssim tensorrt==10.9.0.34
    !sudo sed -i '/developer\.download\.nvidia\.com\/compute\/cuda/s/^/#/' /etc/apt/sources.list.d/*.list
    !wget https://developer.nvidia.com/downloads/compute/machine-learning/tensorrt/10.9.0/local_repo/nv-tensorrt-local-repo-ubuntu2204-10.9.0-cuda-12.8_1.0-1_amd64.deb
    !sudo dpkg -i nv-tensorrt-local-repo-ubuntu2204-10.9.0-cuda-12.8_1.0-1_amd64.deb
    !sudo cp /var/nv-tensorrt-local-repo-ubuntu2204-10.9.0-cuda-12.8/*-keyring.gpg /usr/share/keyrings/
    !echo "deb [signed-by=/usr/share/keyrings/nv-tensorrt-local-AD7406A2-keyring.gpg] \
    file:/var/nv-tensorrt-local-repo-ubuntu2204-10.9.0-cuda-12.8 ./" | \
    sudo tee /etc/apt/sources.list.d/tensorrt-local.list > /dev/null
    !sudo apt update
    !sudo apt install -y --allow-downgrades tensorrt=10.9.0* \
    libnvinfer10=10.9.0.* \
    libnvinfer-plugin10=10.9.0.* \
    libnvinfer-vc-plugin10=10.9.0.* \
    libnvinfer-lean10=10.9.0.* \
    libnvinfer-dispatch10=10.9.0.* \
    libnvonnxparsers10=10.9.0.* \
    python3-libnvinfer=10.9.0.* \
    tensorrt-libs=10.9.0.* """

Library Import

In [ ]:
import os
import cv2
import time
import math
import glob
import torch
import random
import numpy as np
import torch.nn as nn
from tqdm import tqdm
import onnxruntime as ort
from pytorch_msssim import ssim
import torch.nn.functional as F
import torch.multiprocessing as mp
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor, ToPILImage, Compose, ColorJitter

Training Config & Setting

In [ ]:
UPSCALE_FACTOR = 2
BATCH_SIZE = 256
NUM_WORKER = os.cpu_count()
EPOCHS = 150
PATCH_SIZE = 256
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LOSS_ALPHA = 0.84
BASE_DIR = Path(".")
DATA_DIR = Path("/kaggle/input/datasetss")
OUTPUTS_DIR = BASE_DIR / "outputs"
CHECKPOINTS_DIR = BASE_DIR / "checkpoints"
CHECKPOINT_FILE = CHECKPOINTS_DIR / f"best_espcn_x{UPSCALE_FACTOR}.pth"

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        print("Enabling cuDNN benchmark mode for GPU.")
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

set_seed()

Environment Preparation

In [ ]:
def prepare_environment_and_datasets(patch_size):
    CHECKPOINTS_DIR.mkdir(exist_ok=True)
    OUTPUTS_DIR.mkdir(exist_ok=True)

    # Define a path for the cache file
    CACHE_FILE_TRAIN = OUTPUTS_DIR / "valid_train_paths.txt"

    # --- Handle Validation Paths (always scanned, it's fast) ---
    valid_div2k_dir = DATA_DIR / "DIV2K_valid_HR"
    validation_image_paths = glob.glob(str(valid_div2k_dir / '*.*'))

    # --- Handle Training Paths (Check cache first) ---
    valid_train_paths = []
    if CACHE_FILE_TRAIN.exists():
        print(f"Loading cached training image paths from {CACHE_FILE_TRAIN}...")
        try:
            with open(CACHE_FILE_TRAIN, 'r') as f:
                valid_train_paths = [line.strip() for line in f if line.strip()]
            print(f"Loaded {len(valid_train_paths)} paths from cache.")
        except Exception as e:
            print(f"Warning: Could not read cache file {e}. Re-building...")
            valid_train_paths = [] # Ensure list is empty to trigger re-build

    if not valid_train_paths:
        print(f"Cache file not found or was invalid. Building new image list...")
        train_div2k_dir = DATA_DIR / "DIV2K_train_HR"
        train_flickr2k_dir = DATA_DIR / "Flickr2K_HR"
        personal_dir = DATA_DIR / "Personal_HR"

        train_image_paths = glob.glob(str(train_div2k_dir / '*.*')) + \
                            glob.glob(str(train_flickr2k_dir / '*.*'))

        if personal_dir.exists():
            print("Personal dataset found. Adding to training set.")
            train_image_paths += glob.glob(str(personal_dir / '*.*'))
        else:
            print("Personal dataset not found. Proceeding without it.")

        print(f"Found {len(train_image_paths)} potential training images.")
        print(f"Verifying image dimensions against patch size {patch_size}...")

        def is_image_large_enough(image_path, min_size):
            try:
                img = cv2.imread(str(image_path))
                if img is None:
                    print(f"Warning: Failed to load {image_path}. Skipping.")
                    return False
                h, w = img.shape[:2]
                return h >= min_size and w >= min_size
            except Exception as e:
                print(f"Warning: Error reading {image_path}: {e}. Skipping.")
                return False

        valid_train_paths = [
            p for p in tqdm(train_image_paths, desc="Filtering train images")
            if is_image_large_enough(p, patch_size)
        ]

        print(f"Filtered training set: {len(valid_train_paths)} of {len(train_image_paths)} images remain.")

        # Write the new valid paths to the cache file
        try:
            print(f"Saving new cache file to {CACHE_FILE_TRAIN}...")
            with open(CACHE_FILE_TRAIN, 'w') as f:
                for path in valid_train_paths:
                    f.write(f"{path}\n")
        except Exception as e:
            print(f"Warning: Could not write cache file: {e}")

    if not valid_train_paths or not validation_image_paths:
        print("\n--- VERIFICATION FAILED: No images found. Check your './datasets/' folder structure. ---")

    print(f"--- Dataset paths prepared. Train: {len(valid_train_paths)}, Val: {len(validation_image_paths)} ---")
    return valid_train_paths, validation_image_paths

Loss Function

In [ ]:
class CombinedLoss(nn.Module):
    def __init__(self, alpha=0.7):
        super(CombinedLoss, self).__init__()
        self.alpha = alpha
        self.mse_loss = nn.MSELoss()
        try:
            self.ssim_loss_fn = ssim
        except ImportError:
            raise ImportError("Please install pytorch-msssim: pip install pytorch-msssim")

    def forward(self, output, target):
        # MSE Loss
        loss_mse = self.mse_loss(output, target)

        # SSIM Loss (1 - SSIM)
        ssim_score = self.ssim_loss_fn(output, target, data_range=1.0, size_average=True)
        loss_ssim = 1 - ssim_score

        # Combined Loss
        total_loss = self.alpha * loss_mse + (1 - self.alpha) * loss_ssim

        return total_loss, loss_mse, ssim_score

def psnr(mse):
    return 10 * math.log10(1 / mse) if mse > 0 else float('inf')

Create Patches From HR Images

In [ ]:
class TrainingSuperResolutionDataset(Dataset):
    def __init__(self, image_filenames, crop_size, upscale_factor):
        super(TrainingSuperResolutionDataset, self).__init__()
        self.image_filenames = image_filenames
        self.crop_size = crop_size - (crop_size % upscale_factor)
        self.upscale_factor = upscale_factor
        self.to_tensor = ToTensor()
        self.scale_factor = 1 / upscale_factor
        self.transform = Compose([
            ToPILImage(),
            ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            ToTensor()
        ])

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        try:
            hr_image = cv2.imread(self.image_filenames[index])
            if hr_image is None:
                raise IOError(f"cv2.imread failed to load image: {self.image_filenames[index]}")
            hr_image = cv2.cvtColor(hr_image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(f"Error loading image {self.image_filenames[index]}: {e}. Skipping.")
            return self.__getitem__((index + 1) % len(self))

        h, w = hr_image.shape[:2]

        i = random.randint(0, h - self.crop_size)
        j = random.randint(0, w - self.crop_size)
        hr_patch = hr_image[i:i+self.crop_size, j:j+self.crop_size, :]

        if torch.rand(1) > 0.5:
            hr_patch = cv2.flip(hr_patch, 1)

        if torch.rand(1) > 0.5:
            angle = random.choice([cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_180, cv2.ROTATE_90_COUNTERCLOCKWISE])
            hr_patch = cv2.rotate(hr_patch, angle)

        # --- START: Recommended Change ---

        # 1. Create the CLEAN HR (target) tensor first
        hr_patch_tensor = self.transform(hr_patch)

        # 2. Create the CLEAN LR tensor from the clean HR tensor
        lr_patch_tensor = F.interpolate(
            hr_patch_tensor.unsqueeze(0),
            scale_factor=self.scale_factor,
            mode='bicubic',
            align_corners=False,
            antialias=True
        ).squeeze(0)

        # --- END: Recommended Change ---

        return lr_patch_tensor, hr_patch_tensor # (Augmented_LR, Clean_HR)

class ValidationSuperResolutionDataset(Dataset):
    def __init__(self, image_filenames, upscale_factor):
        super(ValidationSuperResolutionDataset, self).__init__()
        self.image_filenames = image_filenames
        self.upscale_factor = upscale_factor
        self.to_tensor = ToTensor()
        self.scale_factor = 1 / upscale_factor

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        try:
            hr_image = cv2.imread(self.image_filenames[index])
            if hr_image is None:
                raise IOError(f"cv2.imread failed to load image: {self.image_filenames[index]}")
            hr_image = cv2.cvtColor(hr_image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(f"Error loading image {self.image_filenames[index]}: {e}. Skipping.")
            return self.__getitem__((index + 1) % len(self))

        h, w = hr_image.shape[:2]
        w_new, h_new = w - (w % self.upscale_factor), h - (h % self.upscale_factor)
        hr_image = hr_image[:h_new, :w_new, :]

        hr_tensor = self.to_tensor(hr_image)

        # This is the "interpolasi bikubik" method from your proposal 
        lr_tensor = F.interpolate(
            hr_tensor.unsqueeze(0),
            scale_factor=self.scale_factor,
            mode='bicubic',
            align_corners=False,
            antialias=True
        ).squeeze(0)
        
        return lr_tensor, hr_tensor

Model Defintion

In [ ]:
class ESPCN(nn.Module):
    def __init__(self, upscale_factor):
        super(ESPCN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 3 * (upscale_factor ** 2), kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)
        self.relu = nn.ReLU() 

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pixel_shuffle(self.conv3(x))
        return x

automatically create ddp_worker.py

In [ ]:
ddp_worker_code = """
# --- Library Import ---

import os
import cv2
import time
import math
import torch
import random
import numpy as np
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
from pathlib import Path
import torch.optim as optim
from pytorch_msssim import ssim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.distributed as dist
from torch.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data.distributed import DistributedSampler
from torchvision.transforms import ToTensor, ToPILImage, Compose, ColorJitter

# --- Training Config & Setting ---

UPSCALE_FACTOR = 2
BATCH_SIZE = 128 # Per GPU
NUM_WORKER = os.cpu_count() // 2 # Per Process
EPOCHS = 150
PATCH_SIZE = 256
LEARNING_RATE = 1e-4
LOSS_ALPHA = 0.84
BASE_DIR = Path(".") 
OUTPUTS_DIR = BASE_DIR / "outputs"
CHECKPOINTS_DIR = BASE_DIR / "checkpoints"
CHECKPOINT_FILE = CHECKPOINTS_DIR / f"best_espcn_x{UPSCALE_FACTOR}.pth"

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        # Note: This print will appear once per process
        print(f"Rank {dist.get_rank() if dist.is_initialized() else 0}: Enabling cuDNN benchmark mode for GPU.")
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# --- Loss Function ---

class CombinedLoss(nn.Module):
    def __init__(self, alpha=0.7):
        super(CombinedLoss, self).__init__()
        self.alpha = alpha
        self.mse_loss = nn.MSELoss()
        try:
            self.ssim_loss_fn = ssim
        except ImportError:
            raise ImportError("Please install pytorch-msssim: pip install pytorch-msssim")

    def forward(self, output, target):
        # MSE Loss
        loss_mse = self.mse_loss(output, target)

        # SSIM Loss (1 - SSIM)
        ssim_score = self.ssim_loss_fn(output, target, data_range=1.0, size_average=True)
        loss_ssim = 1 - ssim_score

        # Combined Loss
        total_loss = self.alpha * loss_mse + (1 - self.alpha) * loss_ssim

        return total_loss, loss_mse, ssim_score

def psnr(mse):
    return 10 * math.log10(1 / mse) if mse > 0 else float('inf')

# --- Create Patches From HR Images ---

class TrainingSuperResolutionDataset(Dataset):
    def __init__(self, image_filenames, crop_size, upscale_factor):
        super(TrainingSuperResolutionDataset, self).__init__()
        self.image_filenames = image_filenames
        self.crop_size = crop_size - (crop_size % upscale_factor)
        self.upscale_factor = upscale_factor
        self.to_tensor = ToTensor()
        self.scale_factor = 1 / upscale_factor
        self.transform = Compose([
            ToPILImage(),
            ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            ToTensor()
        ])

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        try:
            hr_image = cv2.imread(self.image_filenames[index])
            if hr_image is None:
                raise IOError(f"cv2.imread failed to load image: {self.image_filenames[index]}")
            hr_image = cv2.cvtColor(hr_image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(f"Error loading image {self.image_filenames[index]}: {e}. Skipping.")
            # Recursively call __getitem__ with the next index, wrapping around
            return self.__getitem__((index + 1) % len(self))

        h, w = hr_image.shape[:2]

        # Ensure image is large enough (should have been pre-filtered, but good safety check)
        if h < self.crop_size or w < self.crop_size:
             print(f"Warning: Image {self.image_filenames[index]} is smaller than crop size. Skipping.")
             return self.__getitem__((index + 1) % len(self))

        i = random.randint(0, h - self.crop_size)
        j = random.randint(0, w - self.crop_size)
        hr_patch = hr_image[i:i+self.crop_size, j:j+self.crop_size, :]

        # Data Augmentation
        if torch.rand(1) > 0.5:
            hr_patch = cv2.flip(hr_patch, 1) # Horizontal flip

        if torch.rand(1) > 0.5:
            angle = random.choice([cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_180, cv2.ROTATE_90_COUNTERCLOCKWISE])
            hr_patch = cv2.rotate(hr_patch, angle) # Rotations

        # 1. Create the CLEAN HR (target) tensor first, applying color jitter
        hr_patch_tensor = self.transform(hr_patch)

        # 2. Create the CLEAN LR tensor from the clean HR tensor
        lr_patch_tensor = F.interpolate(
            hr_patch_tensor.unsqueeze(0),
            scale_factor=self.scale_factor,
            mode='bicubic',
            align_corners=False,
            antialias=True
        ).squeeze(0)

        return lr_patch_tensor, hr_patch_tensor # (Augmented_LR, Clean_HR)

class ValidationSuperResolutionDataset(Dataset):
    def __init__(self, image_filenames, upscale_factor):
        super(ValidationSuperResolutionDataset, self).__init__()
        self.image_filenames = image_filenames
        self.upscale_factor = upscale_factor
        self.to_tensor = ToTensor()
        self.scale_factor = 1 / upscale_factor

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        try:
            hr_image = cv2.imread(self.image_filenames[index])
            if hr_image is None:
                raise IOError(f"cv2.imread failed to load image: {self.image_filenames[index]}")
            hr_image = cv2.cvtColor(hr_image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(f"Error loading image {self.image_filenames[index]}: {e}. Skipping.")
            return self.__getitem__((index + 1) % len(self))

        h, w = hr_image.shape[:2]
        # Crop to be divisible by upscale_factor
        w_new, h_new = w - (w % self.upscale_factor), h - (h % self.upscale_factor)
        hr_image = hr_image[:h_new, :w_new, :]

        hr_tensor = self.to_tensor(hr_image)

        # Create LR tensor using bicubic downsampling
        lr_tensor = F.interpolate(
            hr_tensor.unsqueeze(0),
            scale_factor=self.scale_factor,
            mode='bicubic',
            align_corners=False,
            antialias=True
        ).squeeze(0)
        
        return lr_tensor, hr_tensor

# --- Model Definition ---

class ESPCN(nn.Module):
    def __init__(self, upscale_factor):
        super(ESPCN, self).__init__()
        # Kaggel notebook had 32, 16. The written file had 64, 64. Using 64, 64 as in the file.
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 3 * (upscale_factor ** 2), kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)
        self.relu = nn.ReLU() 

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pixel_shuffle(self.conv3(x))
        return x

# --- Main Worker Function ---

def main_worker(rank, world_size, train_paths, val_paths):
    DEVICE = torch.device(f'cuda:{rank}')
    torch.cuda.set_device(DEVICE)
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '42069' # Ensure this port is free
    
    # Increase timeout if needed, e.g., for slow startup
    # import datetime
    # timeout_delta = datetime.timedelta(minutes=30)
    # dist.init_process_group("nccl", rank=rank, world_size=world_size, timeout=timeout_delta)
    
    dist.init_process_group("nccl", rank=rank, world_size=world_size)
    set_seed(42 + rank) # Ensure different seed per process

    if rank == 0: print(f"Rank {rank}: Process group initialized.")

    train_dataset = TrainingSuperResolutionDataset(train_paths, crop_size=PATCH_SIZE, upscale_factor=UPSCALE_FACTOR)
    val_dataset = ValidationSuperResolutionDataset(val_paths, upscale_factor=UPSCALE_FACTOR)
    train_sampler = DistributedSampler(train_dataset, num_replicas=world_size, rank=rank, shuffle=True, seed=42)
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False, # Sampler handles shuffle
        num_workers=min(4, NUM_WORKER), # Use fewer workers if CPU is limited
        pin_memory=True, # Pin memory for faster GPU transfer
        persistent_workers=(NUM_WORKER > 0 and min(4, NUM_WORKER) > 0), 
        worker_init_fn=seed_worker,
        sampler=train_sampler, 
        drop_last=True # Important for DDP
    )
    
    # Validation is only done on Rank 0
    if rank == 0:
        val_loader = DataLoader(
            val_dataset, batch_size=1, shuffle=False, 
            num_workers=min(4, NUM_WORKER),
            pin_memory=True,
            persistent_workers=(NUM_WORKER > 0 and min(4, NUM_WORKER) > 0)
        )
        print(f"Rank {rank}: DataLoaders created.")

    model = ESPCN(upscale_factor=UPSCALE_FACTOR).to(DEVICE)
    model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[rank], find_unused_parameters=False)
    
    if rank == 0: print(f"Rank {rank}: Model wrapped in DDP.")

    criterion = CombinedLoss(alpha=LOSS_ALPHA).to(DEVICE)
    # Scale LR by world size
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE * world_size)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
    scaler = GradScaler(enabled=True) # Enable AMP

    start_epoch = 0
    best_psnr = 0.0
    log_file_path = OUTPUTS_DIR / 'training_log.csv'
    training_log = []

    # Load checkpoint only on Rank 0
    if rank == 0:
        CHECKPOINTS_DIR.mkdir(parents=True, exist_ok=True)
        OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)
        if CHECKPOINT_FILE.exists():
            print(f"Rank {rank}: Resuming from checkpoint: {CHECKPOINT_FILE}")
            checkpoint = torch.load(CHECKPOINT_FILE, map_location=DEVICE)

            raw_state_dict = checkpoint['model_state_dict']

            # Handle potential '_orig_mod.' from torch.compile
            needs_compile_strip = any(key.startswith('_orig_mod.') for key in raw_state_dict.keys())
            if needs_compile_strip:
                 print("  Stripping '_orig_mod.' prefix from checkpoint...")
                 stripped_dict = {k.replace('_orig_mod.', ''): v for k, v in raw_state_dict.items()}
                 raw_state_dict = stripped_dict

            model.module.load_state_dict(raw_state_dict)

            try: 
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
            except KeyError:
                 print("Warning: Optimizer/Scheduler state not found in checkpoint. Reinitializing.")

            start_epoch = checkpoint.get('epoch', 0) + 1
            best_psnr = checkpoint.get('best_psnr', 0.0)
            print(f"Rank {rank}: Resumed from epoch {start_epoch}, best PSNR {best_psnr:.4f} dB.")

            if log_file_path.exists():
                try:
                    log_df = pd.read_csv(log_file_path)
                    log_df = log_df[log_df['epoch'] < start_epoch]
                    training_log = log_df.to_dict('records')
                    print(f"Rank {rank}: Loaded {len(training_log)} previous log entries.")
                except Exception as e:
                    print(f"Rank {rank}: Warn: Log load failed: {e}. Starting empty log.")
                    training_log = []
            print(f"Rank {rank}: Checkpoint loaded successfully.")
        else:
            print(f"Rank {rank}: No checkpoint found. Starting fresh.")
            if log_file_path.exists():
                print(f"Rank {rank}: Deleting old log file: {log_file_path}")
                log_file_path.unlink()
            training_log = []

    # --- Synchronization ---
    if rank == 0: print(f"Rank {rank}: Starting synchronization...")

    # Sync start_epoch and best_psnr from rank 0 to others
    sync_list = [start_epoch, best_psnr]
    dist.broadcast_object_list(sync_list, src=0)
    start_epoch, best_psnr = sync_list[0], sync_list[1]

    # Sync model parameters from rank 0
    for param in model.parameters():
        dist.broadcast(param.data, src=0)

    # ========== START: CORRECTED OPTIMIZER SYNC ==========
    # Sync optimizer state (This is the correct way)
    # 1. All ranks create a list. On rank 0, it contains the loaded state.
    #    On other ranks, it's just a placeholder based on their empty optimizer.
    optimizer_state_list = [optimizer.state_dict()]
    
    # 2. Broadcast the list from rank 0 to all other ranks.
    dist.broadcast_object_list(optimizer_state_list, src=0)
    
    # 3. All processes (including rank 0, harmlessly) load the state
    #    that just came from rank 0.
    optimizer.load_state_dict(optimizer_state_list[0])
    # ========== END: CORRECTED OPTIMIZER SYNC ==========

    # Sync scheduler state (last_epoch is crucial)
    scheduler_state_list = [scheduler.state_dict()]
    dist.broadcast_object_list(scheduler_state_list, src=0)
    scheduler.load_state_dict(scheduler_state_list[0])

    if rank == 0: print("Rank 0: All processes synchronized.")
    dist.barrier() # Ensure everyone is ready
    if rank == 0: print(f"Rank {rank}: Passed synchronization barrier.")

    # --- torch.compile ---
    compiled_model = None
    try:
        if rank == 0: print(f"Rank {rank}: Attempting torch.compile...")
        compiled_model = torch.compile(model)
        if rank == 0: print(f"Rank {rank}: torch.compile successful.")
    except Exception as e:
        if rank == 0: print(f"Rank {rank}: torch.compile() failed: {e}. Running eager.")
        compiled_model = model # Fallback to non-compiled model

    model_to_train = compiled_model

    if rank == 0:
        base_model = model_to_train.module
        if hasattr(base_model, '_orig_mod'):
             base_model = base_model._orig_mod
        num_params = sum(p.numel() for p in base_model.parameters() if p.requires_grad)
        print(f"--- Training ESPCN Model ({num_params:,} parameters) ---")

    # --- Training Loop ---
    start_time = time.time()
    if rank == 0: print(f"Rank {rank}: Starting training loop from epoch {start_epoch + 1}...")

    for epoch in range(start_epoch, EPOCHS):
        model_to_train.train()
        train_sampler.set_epoch(epoch)

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", disable=(rank != 0))
        epoch_losses = [] 

        for batch_idx, (lr_images, hr_images) in enumerate(progress_bar):
            lr_images = lr_images.to(DEVICE, non_blocking=True)
            hr_images = hr_images.to(DEVICE, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)

            with autocast(device_type=DEVICE.type, enabled=True):
                outputs = model_to_train(lr_images)
                total_loss, mse, ssim_score = criterion(outputs.float(), hr_images.float())

            if torch.isnan(total_loss):
                print(f"Rank {rank}, Epoch {epoch+1}, Batch {batch_idx}: NaN loss detected! Skipping batch.")
                continue 

            scaler.scale(total_loss).backward()
            
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model_to_train.parameters(), max_norm=1.0)

            scaler.step(optimizer)
            scaler.update()

            if rank == 0:
                epoch_losses.append(total_loss.item())
                progress_bar.set_postfix(
                    Loss=f"{total_loss.item():.4f}",
                    MSE=f"{mse.item():.4f}",
                    SSIM=f"{ssim_score.item():.4f}",
                    LR=f"{scheduler.get_last_lr()[0]:.1E}"
                )

        # --- Validation (Rank 0) ---
        avg_val_psnr, avg_val_ssim = 0.0, 0.0
        needs_save = False 

        if rank == 0:
            model_to_train.eval()
            val_psnr_list = []
            val_ssim_list = []

            with torch.no_grad():
                val_progress = tqdm(val_loader, desc=f"Epoch {epoch+1} Validation", leave=False)
                for i, (lr_val, hr_val) in enumerate(val_progress):
                    lr_val, hr_val = lr_val.to(DEVICE), hr_val.to(DEVICE)

                    eval_model = model_to_train.module
                    if hasattr(eval_model, '_orig_mod'):
                         eval_model = eval_model._orig_mod

                    with autocast(device_type=DEVICE.type, enabled=True):
                        output_val = eval_model(lr_val)
                    
                    output_val = output_val.float() # Ensure float32 for metric calculation
                    hr_val = hr_val.float()

                    mse_val = F.mse_loss(output_val, hr_val)
                    if not torch.isnan(mse_val):
                        val_psnr_list.append(psnr(mse_val.item()))
                    val_ssim_list.append(ssim(output_val, hr_val, data_range=1.0, size_average=True).item())

                    if i == 0 and (epoch + 1) % 5 == 0:
                        try:
                            # Clamp output for image saving
                            sr_pil = ToPILImage()(output_val.squeeze(0).cpu().clamp(0.0, 1.0))
                            output_path = OUTPUTS_DIR / f'val_epoch_{epoch+1}.png'
                            sr_pil.save(output_path)
                        except Exception as img_e:
                             print(f"Rank {rank}: Failed to save validation image: {img_e}")

            avg_val_psnr = np.mean(val_psnr_list) if val_psnr_list else 0.0
            avg_val_ssim = np.mean(val_ssim_list) if val_ssim_list else 0.0
            avg_epoch_loss = np.mean(epoch_losses) if epoch_losses else 0.0

            print(f"\\nEpoch {epoch+1} Summary (Rank 0):")
            print(f"  - Avg Train Loss: {avg_epoch_loss:.4f}")
            print(f"  - Val PSNR: {avg_val_psnr:.4f} dB | Val SSIM: {avg_val_ssim:.4f}")
            print(f"  - Current LR: {scheduler.get_last_lr()[0]:.6f}")

            training_log.append({ 'epoch': epoch + 1, 'train_loss': avg_epoch_loss, 'val_psnr': avg_val_psnr, 'val_ssim': avg_val_ssim, 'learning_rate': scheduler.get_last_lr()[0] })
            try:
                log_df = pd.DataFrame(training_log); log_df.to_csv(log_file_path, index=False)
            except Exception as log_e:
                 print(f"Rank {rank}: Failed to save log: {log_e}")

            if avg_val_psnr > best_psnr:
                best_psnr = avg_val_psnr
                needs_save = True 
                print(f"  - New best model! PSNR: {best_psnr:.4f} dB. Preparing to save...")

        # --- Checkpoint Saving (Rank 0, if needed) ---
        save_info = [needs_save, best_psnr]
        dist.broadcast_object_list(save_info, src=0)
        needs_save, best_psnr = save_info[0], save_info[1] # Update best_psnr on all ranks

        if needs_save and rank == 0:
             print(f"  Rank {rank}: Saving checkpoint for epoch {epoch+1}...") # Corrected epoch num
             try:
                 save_model = model_to_train.module
                 if hasattr(save_model, '_orig_mod'):
                      save_model = save_model._orig_mod

                 model_state_to_save = save_model.state_dict()

                 torch.save({
                     'epoch': epoch, 'model_state_dict': model_state_to_save,
                     'optimizer_state_dict': optimizer.state_dict(),
                     'scheduler_state_dict': scheduler.state_dict(), 'best_psnr': best_psnr,
                 }, CHECKPOINT_FILE)
                 print(f"  Rank {rank}: Checkpoint saved successfully.")
             except Exception as save_e:
                  print(f"Rank {rank}: ERROR saving checkpoint: {save_e}")

        scheduler.step()
        dist.barrier()

    # --- End of Training ---
    dist.barrier() 
    if rank == 0:
        total_time = time.time() - start_time
        print(f"\\nTraining complete in {total_time/60:.2f} minutes ({total_time/3600:.2f} hours).")
        print(f"Best model saved to '{CHECKPOINT_FILE}' (PSNR: {best_psnr:.4f} dB).")
        print(f"Log saved to {log_file_path}")

        try:
             print("Generating final training graphs...")
             log_df = pd.DataFrame(training_log)
             if not log_df.empty:
                 plt.figure(figsize=(12, 5))
                 plt.subplot(1, 2, 1); plt.plot(log_df['epoch'], log_df['val_psnr'], marker='o'); plt.title('Validation PSNR vs. Epoch'); plt.xlabel('Epoch'); plt.ylabel('PSNR (dB)'); plt.grid(True); plt.tight_layout()
                 plt.subplot(1, 2, 2); plt.plot(log_df['epoch'], log_df['val_ssim'], marker='o', color='g'); plt.title('Validation SSIM vs. Epoch'); plt.xlabel('Epoch'); plt.ylabel('SSIM'); plt.grid(True); plt.tight_layout()
                 plt.savefig(OUTPUTS_DIR / 'training_plots.png')
             else:
                  print("Log DataFrame is empty, skipping plot generation.")
        except Exception as plot_e:
             print(f"Error generating plots: {plot_e}")

    dist.destroy_process_group()
"""

# Write the code to the file
try:
    with open("ddp_worker.py", "w") as f:
        f.write(ddp_worker_code)
    print("Successfully wrote ddp_worker.py")
except Exception as e:
    print(f"Error writing ddp_worker.py: {e}")


Execute

In [ ]:
# --- Import the worker function from the file we just created ---
try:
    from ddp_worker import main_worker
    print("Successfully imported main_worker from ddp_worker.py")
except ImportError as e:
    print(f"Could not import main_worker: {e}")
    # Optionally raise an error or exit if import fails
    raise ImportError("Failed to load the worker function. Check ddp_worker.py.") from e

# --- The main_worker function definition is REMOVED from this cell ---

if __name__ == "__main__":
    # Ensure multiprocessing start method is appropriate for CUDA (forkserver or spawn recommended)
    # Kaggle might default correctly, but explicitly setting can help avoid issues.
    try:
        mp.set_start_method('spawn', force=True) # 'spawn' is generally safer with CUDA
        print("Set multiprocessing start method to 'spawn'.")
    except RuntimeError:
        print("Multiprocessing context already set.") # Ignore if already set

    # 1. Prepare datasets *once* in the main process
    prep_start_time = time.time()
    print("--- Preparing Datasets ---")
    # Make sure prepare_environment_and_datasets is defined in a cell ABOVE this one
    train_paths, val_paths = prepare_environment_and_datasets(PATCH_SIZE)
    if not train_paths or not val_paths:
        raise RuntimeError("Dataset paths not found. Halting execution.")
    print(f"--- Dataset preparation complete in {time.time() - prep_start_time:.2f} seconds ---")

    # 2. Check for GPU and decide on world_size
    world_size = torch.cuda.device_count()
    if world_size > 1:
        print(f"Found {world_size} GPUs. Spawning DDP processes...")
        # Note: Pass paths as absolute strings to avoid issues in spawned processes
        abs_train_paths = [str(Path(p).resolve()) for p in train_paths]
        abs_val_paths = [str(Path(p).resolve()) for p in val_paths]
        
        process_context = mp.spawn(
            main_worker,
            args=(world_size, abs_train_paths, abs_val_paths), # Pass absolute paths
            nprocs=world_size,
            join=False # Set join=False initially to potentially catch errors sooner
        )
        try:
            # Wait for processes to complete
            process_context.join()
            print("All DDP processes finished.")
            # Check exit codes if needed (0 means success)
            # for i, exitcode in enumerate(process_context.exitcodes):
            #     if exitcode != 0:
            #         print(f"Warning: Process {i} exited with code {exitcode}")

        except KeyboardInterrupt:
            print("KeyboardInterrupt received, attempting to terminate processes...")
            # Handle manual interruption if needed
            for p in process_context.processes:
                if p.is_alive():
                    p.terminate()
            process_context.join() # Wait for termination
            print("Processes terminated.")
        except Exception as spawn_e:
             print(f"An error occurred during DDP execution: {spawn_e}")
             # Ensure processes are cleaned up on error
             for p in process_context.processes:
                 if p.is_alive():
                     p.terminate()
             process_context.join()
             raise spawn_e # Re-raise the exception

    elif world_size == 1:
        print("Found 1 GPU. Running in single-process mode (no spawn needed).")
        # Run directly in the main process for simplicity if only 1 GPU
        try:
             # Pass paths directly
             main_worker(0, 1, train_paths, val_paths)
        except KeyboardInterrupt:
             print("Training interrupted.")
        except Exception as e:
             print(f"An error occurred during single-GPU execution: {e}")
             raise e
    else:
        print("No GPU found. Running on CPU (not recommended).")
        try:
             main_worker(0, 1, train_paths, val_paths) # Will run on CPU
        except KeyboardInterrupt:
             print("Training interrupted.")
        except Exception as e:
             print(f"An error occurred during CPU execution: {e}")
             raise e

Convert To FP16

In [ ]:
print("--- Applying FP16 Quantization ---")

quant_model = ESPCN(upscale_factor=UPSCALE_FACTOR).to(DEVICE)
checkpoint_path = CHECKPOINTS_DIR / f"best_espcn_x{UPSCALE_FACTOR}.pth"
checkpoint = torch.load(CHECKPOINT_FILE, map_location=DEVICE)

# <<<--- ADD THIS BLOCK --- >>>
original_state_dict = checkpoint['model_state_dict']
new_state_dict = {}
needs_prefix_stripping = any(key.startswith('_orig_mod.') for key in original_state_dict.keys())
if needs_prefix_stripping:
    print("Detected '_orig_mod.' prefix from torch.compile(). Stripping prefix for FP16 conversion...")
    for key, value in original_state_dict.items():
        if key.startswith('_orig_mod.'):
            new_key = key[len('_orig_mod.'):] # Remove the prefix
            new_state_dict[new_key] = value
        else:
            new_state_dict[key] = value # Keep keys without the prefix as is
else:
     print("No '_orig_mod.' prefix detected. Loading state dict as is for FP16 conversion.")
     new_state_dict = original_state_dict # Use the original if no prefix found

quant_model.load_state_dict(new_state_dict) # <<<--- LOAD THE STRIPPED DICT
# <<<--- END OF ADDED BLOCK --- >>>

quant_model.eval()
# Note: .half() is usually for GPU. On CPU, it might be slow or unsupported.
# Consider keeping it FP32 for CPU-only export unless you have specific needs.
# If you keep .half(), ensure subsequent ONNX export/benchmark handles FP16.
quant_model.half()

print("Model converted to FP16.")

quantized_model_path = CHECKPOINTS_DIR / f"best_espcn_x{UPSCALE_FACTOR}_fp16.pth"
# Saving the state_dict directly is correct here
torch.save(quant_model.state_dict(), quantized_model_path)

print(f"FP16 quantized model saved to: {quantized_model_path}")

Export To ONNX

In [ ]:
print("--- Exporting to ONNX ---")

EXPORT_FP16 = True

if EXPORT_FP16:
    print("Exporting FP16 model...")
    onnx_model = ESPCN(upscale_factor=UPSCALE_FACTOR)
    checkpoint_path_fp16 = CHECKPOINTS_DIR / f"best_espcn_x{UPSCALE_FACTOR}_fp16.pth"
    onnx_model.load_state_dict(torch.load(checkpoint_path_fp16))
    onnx_model.eval().half().to(DEVICE)
    dummy_input = torch.randn(1, 3, 720, 1280, device=DEVICE).half()
    onnx_model_path = OUTPUTS_DIR / f"espcn_x{UPSCALE_FACTOR}_fp16.onnx"
else:
    print("Exporting FP32 model...")
    onnx_model = ESPCN(upscale_factor=UPSCALE_FACTOR).to(DEVICE)
    checkpoint_path = CHECKPOINTS_DIR / f"best_espcn_x{UPSCALE_FACTOR}.pth"
    checkpoint = torch.load(CHECKPOINT_FILE, map_location=DEVICE)

    # <<< --- START: ADD THIS FIX --- >>>
    original_state_dict = checkpoint['model_state_dict']
    new_state_dict = {}
    needs_prefix_stripping = any(key.startswith('_orig_mod.') for key in original_state_dict.keys())

    if needs_prefix_stripping:
        print("Detected '_orig_mod.' prefix. Stripping for ONNX export...")
        for key, value in original_state_dict.items():
            if key.startswith('_orig_mod.'):
                new_key = key[len('_orig_mod.'):] # Remove the prefix
                new_state_dict[new_key] = value
            else:
                new_state_dict[key] = value
    else:
         print("No '_orig_mod.' prefix detected. Loading state dict as is for ONNX export.")
         new_state_dict = original_state_dict

    onnx_model.load_state_dict(new_state_dict) # <<< --- LOAD THE STRIPPED DICT
    # <<< --- END: ADD THIS FIX --- >>>

    onnx_model.eval()
    dummy_input = torch.randn(1, 3, 720, 1280, device=DEVICE)
    onnx_model_path = OUTPUTS_DIR / f"espcn_x{UPSCALE_FACTOR}.onnx"

torch.onnx.export(
    onnx_model,
    dummy_input,
    str(onnx_model_path),
    export_params=True,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamo=True,
    dynamic_axes={
        'input': {2: 'height', 3: 'width'},
        'output': {2: 'height_out', 3: 'width_out'}
    }
)

print(f"Model successfully exported to ONNX: {onnx_model_path}")

Benchmark

In [ ]:
print("\n--- Running Inference Benchmark ---")

if EXPORT_FP16:
    test_image_np = np.random.randn(1, 3, 720, 1280).astype(np.float16)
else:
    test_image_np = np.random.randn(1, 3, 720, 1280).astype(np.float32)
test_image_torch = torch.from_numpy(test_image_np).to(DEVICE)

if EXPORT_FP16:
    test_image_torch = test_image_torch.half()

if DEVICE == "cuda":
    torch.cuda.synchronize()

with torch.no_grad():
    _ = onnx_model(test_image_torch)
    if DEVICE == "cuda":
        torch.cuda.synchronize()
    start_time = time.time()
    for _ in range(100):
        _ = onnx_model(test_image_torch)
    if DEVICE == "cuda":
        torch.cuda.synchronize()
    pytorch_time = (time.time() - start_time) / 100

print(f"PyTorch Inference Time: {pytorch_time * 1000:.4f} ms per image")

providers = [
    # ==> Tier 1: Highest-Performance, Hardware-Specific Providers (NVIDIA, AMD, Intel, Apple)
    'TensorrtExecutionProvider',      # NVIDIA's top-tier for speed
    'CUDAExecutionProvider',          # Standard NVIDIA GPU provider
    'MIGraphXExecutionProvider',      # AMD's high-performance graph compiler on Linux
    'ROCmExecutionProvider',          # Standard AMD GPU provider on Linux
    'OpenVINOExecutionProvider',      # Optimized for Intel GPUs and CPUs
    'CoreMLExecutionProvider',        # For Apple M-series chips (macOS, iOS)

    # ==> Tier 2: Specialized NPU/Edge/Mobile Providers
    'QNNExecutionProvider',           # Qualcomm AI Engine Direct (Snapdragon)
    'NNAPIExecutionProvider',         # Android Neural Networks API
    'CANNExecutionProvider',          # Huawei Ascend Chips
    'RockchipNpuExecutionProvider',   # Rockchip NPUs
    'VitisAIExecutionProvider',       # Xilinx FPGAs
    'ArmNNExecutionProvider',         # Arm NN SDK
    'ACLExecutionProvider',           # Arm Compute Library

    # ==> Tier 3: General-Purpose GPU Provider (Windows)
    'DmlExecutionProvider',           # DirectX 12 for NVIDIA, AMD, Intel GPUs on Windows

    # ==> Tier 4: Optimized CPU Providers
    'DnnlExecutionProvider',          # Intel's high-performance DNNL for CPUs
    'XnnpackExecutionProvider',       # Optimized for ARM CPUs

    # ==> Tier 5: Advanced Compiler & Cloud Providers
    'TvmExecutionProvider',           # Apache TVM
    'AzureExecutionProvider',         # For running on Azure

    # ==> Tier 6: Default Fallback
    'CPUExecutionProvider',           # The universal fallback that runs on any CPU
]

session = ort.InferenceSession(str(onnx_model_path), providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
ort_inputs = {session.get_inputs()[0].name: test_image_np}
_ = session.run(None, ort_inputs)
start_time = time.time()
for _ in range(100):
    _ = session.run(None, ort_inputs)
onnx_time = (time.time() - start_time) / 100

print(f"ONNX Runtime Inference Time: {onnx_time * 1000:.4f} ms per image")

if pytorch_time > 0:
    speed_increase = (pytorch_time - onnx_time) / pytorch_time * 100

    print(f"\nResult: ONNX Runtime is ~{speed_increase:.2f}% faster than PyTorch for this model.")